<a id="data-source"></a>
## Modeling -  Employment Growth and Office Real Estate Values  

Chen Chen

### Refer to below for prior steps:
---
[**Data Dictionary**](https://github.com/jchen9619/Employment-Growth-and-Office-Property-Values/blob/master/Data%20Dictionary%20-%20Preprocessed%20Dataset.ipynb) <br>
[**EDA Notebook (Pre-processed Dataset)**](https://github.com/jchen9619/Employment-Growth-and-Office-Property-Values/blob/master/EDA%20-%20Employment%20Growth%20and%20Office%20Property%20Values.ipynb) <br>
[**Feature Engineering Notebook**](https://github.com/jchen9619/Employment-Growth-and-Office-Property-Values/blob/master/Feature%20Engineering%20-%20Employment%20Growth%20and%20Office%20Property%20Values.ipynb) <br>

### Methodology:
---
Each model is run with five-fold cross-validation, on both Untransformed and Transformed datasets. Best output is defined by model with the **lowest RMSE** for each outcome variable.

In [1]:
#Import all modules
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import model_selection
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import itertools
import warnings
import math
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score

warnings.filterwarnings("ignore")

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [3]:
pd.set_option('display.max_colwidth', None)

### Loading preprocessed and log-transformed dataset for modeling: <br>
---


In [4]:
#log-transformed dataset:
Transformed_Dataset=pd.read_excel('/Users/chenmouse/Desktop/GA-Folder/JudyChenGA0316/JC0316/GA Final Project/PostLogTransform.xlsx')

In [5]:
#load preprocessed untransformed dataset
Untransformed_Dataset=pd.read_excel('/Users/chenmouse/Desktop/GA-Folder/JudyChenGA0316/JC0316/GA Final Project/Preprocessed.xlsx')

In [6]:
Transformed_Dataset.name='Log-transformed Dataset'
Untransformed_Dataset.name='Preprocessed Untransformed Dataset'

In [7]:
DS_ls=[Untransformed_Dataset,Transformed_Dataset]

Compile a list of combination of features and output variables to rotate through: 

In [8]:
All_Features= ['TOT_EMP', 'JOBS_1000', 'LOC_QUOTIENT', 'A_MEAN', 'A_MEDIAN']

In [9]:
All_Features_2= [['TOT_EMP', 'JOBS_1000', 'LOC_QUOTIENT', 'A_MEAN', 'A_MEDIAN']]

In [10]:
for i in range(1,5):
    for ft_combo in itertools.combinations(All_Features,i):
        All_Features_2.append(list(ft_combo))

In [11]:
cols=['MSA-OCC','TOT_EMP','JOBS_1000','LOC_QUOTIENT','A_MEAN','A_MEDIAN','OCBD Income Return (%)','OSUB Income Return (%)','Office Income Return (%)','OCBD Apprec Return (%)','OSUB Apprec Return (%)','Office Apprec Return (%)','OCBD Total Return (%)','OSUB Total Return (%)','Office Total Return (%)']

In [12]:
outputs = []
for k in range(6,len(cols)):
    outputs.append(str(cols[k]))
outputs_s=pd.Series(outputs)

### Modeling: 
---
#### Function for Four-Fold Cross Validation, Untuned Models

In [13]:
KF = model_selection.KFold(n_splits=4, shuffle=True, random_state=1)

In [101]:
def FFCVUntuned(dataset, features, regressor): 
    RMSE_4FM_ls=[]
    R2Score_4FM_ls=[]
    ft_ls=[]
    ft_imp_ls=[]
    dataset_ls=[]
    for i in range(6,len(cols)):
        X = dataset[features][dataset[str(cols[i])].notnull()]
        y = dataset[str(cols[i])][dataset[str(cols[i])].notnull()]
        rmse_values = []
        scores = []
        mean_rmse_ls=[]
        mean_r2_ls=[]
        for train_index, test_index in KF.split(X, y):
                model = regressor.fit(X.iloc[train_index], y.iloc[train_index])
                rmse_values.append(np.sqrt(metrics.mean_squared_error(y.iloc[test_index], model.predict(X.iloc[test_index]))))
                scores.append(model.score(X, y))
                mean_rmse=(sum(rmse_values)/len(rmse_values)).round(4)
                mean_r2=(sum(scores)/len(scores)).round(4)
        mean_rmse_ls.append(mean_rmse)
        mean_r2_ls.append(mean_r2)
        RMSE_4FM_ls.append(mean_rmse_ls)
        R2Score_4FM_ls.append(mean_r2_ls)
        if str(regressor)=='LinearRegression()':
            ft_ls.append(str(features))
        else:
            np.argmax(model.feature_importances_)
            ft_ls.append(All_Features[np.argmax(model.feature_importances_)])  
        if str(regressor)=='LinearRegression()':
            ft_imp_ls.append('')
        else:
            ft_imp_ls.append(max(model.feature_importances_).round(2))
        dataset_ls.append(dataset.name)
    if str(regressor)=='LinearRegression()':
        model_output = pd.DataFrame({'Output Variable': outputs_s, '4-Fold Mean RMSE': pd.Series(RMSE_4FM_ls), '4-Fold Mean R2': pd.Series(R2Score_4FM_ls), 'Features': pd.Series(ft_ls), 'Dataset': pd.Series(dataset_ls)})
    else:
        model_output = pd.DataFrame({'Output Variable': outputs_s, '4-Fold Mean RMSE': pd.Series(RMSE_4FM_ls), '4-Fold Mean R2': pd.Series(R2Score_4FM_ls), 'Best Feature': pd.Series(ft_ls), 'Feature Importance': pd.Series(ft_imp_ls), 'Dataset': pd.Series(dataset_ls)})
    return model_output

### Linear Regression
**Log-Transformed Dataset** <br>

**Best features vary for different output variables, includes both job growth (<u>TOT_EMP</u>,  <u>JOBS_1000</u>) and salary metrics (<u>A_MEAN</u>, <u>A_MEDIAN</u>)**

In [102]:
aggLR_Transformed_df=pd.DataFrame()
for ft in All_Features_2:
    to_append=FFCVUntuned(Transformed_Dataset,ft,LinearRegression())
    aggLR_Transformed_df=aggLR_Transformed_df.append(to_append)    

In [103]:
print(color.BOLD + "Best Features for Linear Regression: Log-transformed Dataset" + color.END)
minRMSE_LRT = []
aggLR_Transformed_df_minRMSE=pd.DataFrame()
for i in range(6,len(cols)):
    minRMSE_LRT=aggLR_Transformed_df[aggLR_Transformed_df["Output Variable"]==cols[i]].sort_values("4-Fold Mean RMSE", ascending=True).iloc[0]
    aggLR_Transformed_df_minRMSE=aggLR_Transformed_df_minRMSE.append(minRMSE_LRT, ignore_index=True)
aggLR_Transformed_df_minRMSE=aggLR_Transformed_df_minRMSE[['Output Variable','4-Fold Mean RMSE','4-Fold Mean R2','Features','Dataset']]
aggLR_Transformed_df_minRMSE

Best Features for Linear Regression: Log-transformed Dataset


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Features,Dataset
0,OCBD Income Return (%),[0.582],[0.3427],"['JOBS_1000', 'A_MEAN', 'A_MEDIAN']",Log-transformed Dataset
1,OSUB Income Return (%),[0.6042],[0.2727],"['JOBS_1000', 'A_MEAN']",Log-transformed Dataset
2,Office Income Return (%),[0.6115],[0.3163],"['JOBS_1000', 'A_MEAN', 'A_MEDIAN']",Log-transformed Dataset
3,OCBD Apprec Return (%),[0.7793],[0.5358],"['TOT_EMP', 'A_MEAN', 'A_MEDIAN']",Log-transformed Dataset
4,OSUB Apprec Return (%),[0.9047],[0.407],"['TOT_EMP', 'JOBS_1000', 'A_MEAN']",Log-transformed Dataset
5,Office Apprec Return (%),[0.6412],[0.6066],"['TOT_EMP', 'JOBS_1000', 'A_MEAN']",Log-transformed Dataset
6,OCBD Total Return (%),[0.5864],[0.5039],"['TOT_EMP', 'LOC_QUOTIENT', 'A_MEAN', 'A_MEDIAN']",Log-transformed Dataset
7,OSUB Total Return (%),[0.6078],[0.5305],"['TOT_EMP', 'A_MEAN']",Log-transformed Dataset
8,Office Total Return (%),[0.6078],[0.5305],"['TOT_EMP', 'A_MEAN']",Log-transformed Dataset


#### Preprocessed Untransformed Dataset
**Best features vary for different output variables, but are all job growth (<u>TOT_EMP</u>, <u>JOBS_1000</u>) metrics**

In [104]:
aggLR_Untransformed_df=pd.DataFrame()
for ft in All_Features_2:
    to_append=FFCVUntuned(Untransformed_Dataset,ft,LinearRegression())
    aggLR_Untransformed_df=aggLR_Untransformed_df.append(to_append)

In [105]:
print(color.BOLD + "Best Features for Linear Regression: Untransformed Dataset" + color.END)
minRMSE_LRU = []
aggLR_Untransformed_df_minRMSE=pd.DataFrame()
for i in range(6,len(cols)):
    minRMSE_LRU=aggLR_Untransformed_df[aggLR_Untransformed_df["Output Variable"]==cols[i]].sort_values("4-Fold Mean RMSE", ascending=True).iloc[0]
    aggLR_Untransformed_df_minRMSE=aggLR_Untransformed_df_minRMSE.append(minRMSE_LRU, ignore_index=True)
aggLR_Untransformed_df_minRMSE=aggLR_Untransformed_df_minRMSE[['Output Variable','4-Fold Mean RMSE','4-Fold Mean R2','Features','Dataset']]
aggLR_Untransformed_df_minRMSE

Best Features for Linear Regression: Untransformed Dataset


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Features,Dataset
0,OCBD Income Return (%),[0.1606],[0.0089],['TOT_EMP'],Preprocessed Untransformed Dataset
1,OSUB Income Return (%),[0.2006],[0.025],['TOT_EMP'],Preprocessed Untransformed Dataset
2,Office Income Return (%),[0.1829],[0.0211],['TOT_EMP'],Preprocessed Untransformed Dataset
3,OCBD Apprec Return (%),[0.1974],[0.0975],"['TOT_EMP', 'JOBS_1000']",Preprocessed Untransformed Dataset
4,OSUB Apprec Return (%),[0.1947],[0.1037],"['TOT_EMP', 'JOBS_1000']",Preprocessed Untransformed Dataset
5,Office Apprec Return (%),[0.1836],[0.1032],"['TOT_EMP', 'JOBS_1000']",Preprocessed Untransformed Dataset
6,OCBD Total Return (%),[0.3429],[0.0373],"['TOT_EMP', 'JOBS_1000']",Preprocessed Untransformed Dataset
7,OSUB Total Return (%),[0.3216],[0.0327],"['TOT_EMP', 'JOBS_1000']",Preprocessed Untransformed Dataset
8,Office Total Return (%),[0.3216],[0.0327],"['TOT_EMP', 'JOBS_1000']",Preprocessed Untransformed Dataset


### Linear Regression: Conclusion
#### The model with the lowest RMSE uses <u>TOT_EMP</u> on untransformed dataset. However, it suffers from extremely low R2 scores. Therefore, we consider alternative models.
---

### Decision Tree
**Untuned Model**

In [106]:
agg_output_DT=pd.DataFrame()
for i in range(0, 2):
    to_append_dt=FFCVUntuned(DS_ls[i],All_Features,DecisionTreeRegressor(random_state=1))
    agg_output_DT=agg_output_DT.append(to_append_dt)

In [107]:
print((color.BOLD + "Untuned Decision Tree Output" + color.END))
agg_output_DT.sort_values(by="4-Fold Mean RMSE", ascending=True).reset_index(drop=True)

Untuned Decision Tree Output


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Best Feature,Feature Importance,Dataset
0,Office Apprec Return (%),[0.2288],[0.609],JOBS_1000,0.37,Preprocessed Untransformed Dataset
1,OCBD Income Return (%),[0.2339],[0.4654],TOT_EMP,0.44,Preprocessed Untransformed Dataset
2,OCBD Apprec Return (%),[0.2348],[0.6687],JOBS_1000,0.27,Preprocessed Untransformed Dataset
3,Office Income Return (%),[0.2383],[0.5752],JOBS_1000,0.31,Preprocessed Untransformed Dataset
4,OSUB Income Return (%),[0.2397],[0.6487],TOT_EMP,0.38,Preprocessed Untransformed Dataset
5,OSUB Apprec Return (%),[0.2706],[0.5299],JOBS_1000,0.37,Preprocessed Untransformed Dataset
6,OSUB Total Return (%),[0.3781],[0.6559],JOBS_1000,0.42,Preprocessed Untransformed Dataset
7,Office Total Return (%),[0.3781],[0.6559],JOBS_1000,0.42,Preprocessed Untransformed Dataset
8,OCBD Total Return (%),[0.4472],[0.5747],TOT_EMP,0.43,Preprocessed Untransformed Dataset
9,OCBD Total Return (%),[0.8224],[0.7184],TOT_EMP,0.60,Log-transformed Dataset


**For all output variables, the untransformed dataset produces much lower RMSEs. Next, we examine whether hyperparameter tuning improves the output.**

#### Hyperparameter Tuning 
#### Function for Tuning Models: Four-Fold Cross Validation

In [108]:
def Tune(dataset, regressor, parameters):
    tuning_model=GridSearchCV(regressor,param_grid=parameters,scoring='neg_mean_squared_error',cv=4,verbose=0) 
    RMSE_4FMbp_ls=[]
    R2Score_4FMbp_ls=[]
    max_depth_ls=[]
    max_fts_ls=[]
    min_samples_leaf_ls=[]
    best_ft_ls=[]
    ft_importance_ls=[]
    sec_ft_ls=[]
    sec_ft_imp_ls=[]
    for i in range(6,len(cols)):
        X = dataset[All_Features][dataset[str(cols[i])].notnull()]
        y = dataset[str(cols[i])][dataset[str(cols[i])].notnull()]
        tuning_model.fit(X,y)
        tuned_model_best=tuning_model.best_estimator_
        rmse_values_bp = []
        scores_bp = []
        mean_bp_rmse_ls=[]
        mean_bp_r2_ls=[]
        accuracy_score_bp_ls=[]
        for train_index, test_index in KF.split(X, y):
            model_bp=tuned_model_best.fit(X.iloc[train_index], y.iloc[train_index])
            best_param=tuning_model.best_params_
            rmse_values_bp.append(np.sqrt(metrics.mean_squared_error(y.iloc[test_index], model_bp.predict(X.iloc[test_index]))))
            scores_bp.append(model_bp.score(X, y))
            mean_rmse_bp=(sum(rmse_values_bp)/len(rmse_values_bp)).round(4)
            mean_r2_bp=(sum(scores_bp)/len(scores_bp)).round(4)
        mean_bp_rmse_ls.append(float(mean_rmse_bp))
        mean_bp_r2_ls.append(float(mean_r2_bp))
        RMSE_4FMbp_ls.append(mean_bp_rmse_ls)
        R2Score_4FMbp_ls.append(mean_bp_r2_ls)
        max_depth_ls.append(best_param['max_depth'])
        max_fts_ls.append(best_param['max_features'])       
        min_samples_leaf_ls.append(best_param['min_samples_leaf'])
        bestft=pd.DataFrame({'feature':All_Features, 'importance':tuned_model_best.feature_importances_})
        bestft_sorted=bestft.sort_values(by='importance', ascending=False).reset_index()
        best_ft_ls.append(bestft_sorted.at[0,'feature'])
        ft_importance_ls.append(bestft_sorted.at[0,'importance'])
        sec_ft_ls.append(bestft_sorted.at[1,'feature'])
        sec_ft_imp_ls.append(bestft_sorted.at[1,'importance'])
    max_fts_s=pd.Series(max_fts_ls)
    outputTuned=pd.DataFrame({'Output Variable':outputs_s,'4-Fold Mean RMSE':pd.Series(RMSE_4FMbp_ls), '4-Fold Mean R2': pd.Series(R2Score_4FMbp_ls),
                           'Optimal Depth':pd.Series(max_depth_ls),
                           'Max Features':max_fts_s,
                           'Min. Samples for Leaf Node':pd.Series(min_samples_leaf_ls),
                          'Best Feature':pd.Series(best_ft_ls),
                          'Best Feature Importance':pd.Series(ft_importance_ls),
                           'Second Best Feature': pd.Series(sec_ft_ls),
                           'Second Best Feature Importance': pd.Series(sec_ft_imp_ls)})
    return outputTuned

In [109]:
parametersDT={"splitter":["best","random"],
            "max_depth" : list(range(1,15)),
           "max_features":list(range(1,5)),
           "min_samples_leaf":[3, 5, 7, 10, 15, 20]}   

In [110]:
print((color.BOLD + "Decision Tree: Best Hyperparameters" + color.END))
outputDTTuned=Tune(Untransformed_Dataset, DecisionTreeRegressor(random_state=1), parametersDT)
outputDTTuned

Decision Tree: Best Hyperparameters


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Optimal Depth,Max Features,Min. Samples for Leaf Node,Best Feature,Best Feature Importance,Second Best Feature,Second Best Feature Importance
0,OCBD Income Return (%),[0.1544],[0.3679],4,1,3,JOBS_1000,0.530575,A_MEDIAN,0.254948
1,OSUB Income Return (%),[0.1981],[0.2777],8,3,3,JOBS_1000,0.654004,TOT_EMP,0.192638
2,Office Income Return (%),[0.1822],[0.21],9,3,5,JOBS_1000,0.681575,TOT_EMP,0.161191
3,OCBD Apprec Return (%),[0.2028],[0.1833],6,4,7,JOBS_1000,0.567670,TOT_EMP,0.253719
4,OSUB Apprec Return (%),[0.2001],[0.1616],2,3,10,A_MEDIAN,0.450928,LOC_QUOTIENT,0.395005
5,Office Apprec Return (%),[0.1976],[0.1513],3,4,3,JOBS_1000,0.563959,A_MEAN,0.358448
6,OCBD Total Return (%),[0.3391],[0.1463],6,2,5,JOBS_1000,0.703201,TOT_EMP,0.287287
7,OSUB Total Return (%),[0.3254],[0.145],2,3,7,TOT_EMP,0.788693,A_MEAN,0.211307
8,Office Total Return (%),[0.3254],[0.145],2,3,7,TOT_EMP,0.788693,A_MEAN,0.211307


### Decision Tree Hyperparameter Tuning: Conclusion
#### While RMSEs are slightly reduced with hyperparameter tuning, the R2 is significantly lower than untuned decision tree model.
**The best hyperparameters vary based on the type of real estate return.**

#### Max Features:
Optimal at 3 for 5 out of 9 output variables.

#### Feature Importance: 
Total and Income returns (to a lesser degree) are primarily dependent on features relating to the number of jobs for an occupation, <u>**JOBS_1000**</u> (number of jobs in an occupation per 1,000 jobs in an MSA) and <u>**Tot_EMP**</u>  (estimated total employment ).  Appreciation returns for suburban and total office has a higher reliance on earnings compared to the others, evidenced by importance of <u>**A_MEDIAN**</u> (annual median salary) and  <u>**A_MEAN**</u> (annual mean salary).

#### Ensemble Method: XGBoost
#### Find Best Dataset: Find Best Dataset (lowest RMSE): Original Dataset

In [111]:
agg_output_XG=pd.DataFrame()
for i in range(0, 2):
    to_appendXG=FFCVUntuned(DS_ls[i],All_Features, XGBRegressor(random_state=1))
    agg_output_XG=agg_output_XG.append(to_appendXG)

In [112]:
print((color.BOLD + "Untuned XG-Boost (Decision Tree) Output" + color.END))
agg_output_XG.sort_values(by="4-Fold Mean RMSE", ascending=True).reset_index(drop=True)

Untuned XG-Boost (Decision Tree) Output


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Best Feature,Feature Importance,Dataset
0,OCBD Income Return (%),[0.1689],[0.719],JOBS_1000,0.34,Preprocessed Untransformed Dataset
1,OCBD Apprec Return (%),[0.1825],[0.8013],JOBS_1000,0.35,Preprocessed Untransformed Dataset
2,Office Income Return (%),[0.1907],[0.7268],JOBS_1000,0.34,Preprocessed Untransformed Dataset
3,OSUB Income Return (%),[0.1965],[0.7625],JOBS_1000,0.31,Preprocessed Untransformed Dataset
4,Office Apprec Return (%),[0.2036],[0.6814],JOBS_1000,0.50,Preprocessed Untransformed Dataset
5,OSUB Apprec Return (%),[0.2326],[0.6578],JOBS_1000,0.40,Preprocessed Untransformed Dataset
6,OCBD Total Return (%),[0.3283],[0.7707],JOBS_1000,0.37,Preprocessed Untransformed Dataset
7,Office Total Return (%),[0.3426],[0.6981],JOBS_1000,0.43,Preprocessed Untransformed Dataset
8,OSUB Total Return (%),[0.3426],[0.6981],JOBS_1000,0.43,Preprocessed Untransformed Dataset
9,OCBD Income Return (%),[0.7107],[0.7211],TOT_EMP,0.38,Log-transformed Dataset


### Conclusion: Decision Tree Modeling
**Overall, the best method is Decision Tree with XGBoost. R2 are significantly higher while RMSE remains similar to Decision Tree Model with tuned hyperparameters. This is likely due to the nature of XGBoost learning from previous errors with its gradient boosting framework.**

In [113]:
agg_all_DT=pd.merge(pd.merge(agg_output_DT[0:9], outputDTTuned, on='Output Variable'),agg_output_XG[0:9], on='Output Variable')

In [114]:
agg_all_DT=agg_all_DT[["Output Variable", "4-Fold Mean RMSE_x","4-Fold Mean R2_x", 
            "4-Fold Mean RMSE_y","4-Fold Mean R2_y",
            "4-Fold Mean RMSE","4-Fold Mean R2"]]

In [115]:
agg_all_DT.rename(columns={'Output Variable':'Output Variable', '4-Fold Mean RMSE_x':'DT Untuned: 4-Fold Mean RMSE',
                 '4-Fold Mean R2_x':'DT Untuned: 4-Fold Mean R2',
                 '4-Fold Mean RMSE_y':'DT Tuned: 4-Fold Mean RMSE',
                 '4-Fold Mean R2_y':'DT Tuned: 4-Fold Mean R2',
                 '4-Fold Mean RMSE':'XG Boost: 4-Fold Mean RMSE',
                 '4-Fold Mean R2':'XG Boost:4-Fold Mean R2'}, inplace=True)

In [116]:
agg_all_DT

,Output Variable,DT Untuned: 4-Fold Mean RMSE,DT Untuned: 4-Fold Mean R2,DT Tuned: 4-Fold Mean RMSE,DT Tuned: 4-Fold Mean R2,XG Boost: 4-Fold Mean RMSE,XG Boost:4-Fold Mean R2
0,OCBD Income Return (%),[0.2339],[0.4654],[0.1544],[0.3679],[0.1689],[0.719]
1,OSUB Income Return (%),[0.2397],[0.6487],[0.1981],[0.2777],[0.1965],[0.7625]
2,Office Income Return (%),[0.2383],[0.5752],[0.1822],[0.21],[0.1907],[0.7268]
3,OCBD Apprec Return (%),[0.2348],[0.6687],[0.2028],[0.1833],[0.1825],[0.8013]
4,OSUB Apprec Return (%),[0.2706],[0.5299],[0.2001],[0.1616],[0.2326],[0.6578]
5,Office Apprec Return (%),[0.2288],[0.609],[0.1976],[0.1513],[0.2036],[0.6814]
6,OCBD Total Return (%),[0.4472],[0.5747],[0.3391],[0.1463],[0.3283],[0.7707]
7,OSUB Total Return (%),[0.3781],[0.6559],[0.3254],[0.145],[0.3426],[0.6981]
8,Office Total Return (%),[0.3781],[0.6559],[0.3254],[0.145],[0.3426],[0.6981]


### Random Forest  
**Untuned Model**

In [117]:
agg_output_RF=pd.DataFrame()
for i in range(0, 2):
    to_append_RF=FFCVUntuned(DS_ls[i],All_Features,RandomForestRegressor(random_state=1))
    agg_output_RF=agg_output_RF.append(to_append_RF)

In [118]:
print((color.BOLD + "Untuned Random Forest Output" + color.END))
agg_output_RF.sort_values(by="4-Fold Mean RMSE", ascending=True).reset_index(drop=True)

Untuned Random Forest Output


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Best Feature,Feature Importance,Dataset
0,OCBD Income Return (%),[0.1519],[0.6716],TOT_EMP,0.37,Preprocessed Untransformed Dataset
1,Office Income Return (%),[0.171],[0.6924],TOT_EMP,0.33,Preprocessed Untransformed Dataset
2,Office Apprec Return (%),[0.185],[0.6733],JOBS_1000,0.34,Preprocessed Untransformed Dataset
3,OSUB Income Return (%),[0.1882],[0.698],TOT_EMP,0.36,Preprocessed Untransformed Dataset
4,OCBD Apprec Return (%),[0.1978],[0.6747],JOBS_1000,0.31,Preprocessed Untransformed Dataset
5,OSUB Apprec Return (%),[0.2116],[0.6254],TOT_EMP,0.30,Preprocessed Untransformed Dataset
6,OCBD Total Return (%),[0.3198],[0.6882],JOBS_1000,0.31,Preprocessed Untransformed Dataset
7,Office Total Return (%),[0.3215],[0.6549],JOBS_1000,0.38,Preprocessed Untransformed Dataset
8,OSUB Total Return (%),[0.3215],[0.6549],JOBS_1000,0.38,Preprocessed Untransformed Dataset
9,OSUB Income Return (%),[0.6127],[0.7089],JOBS_1000,0.34,Log-transformed Dataset


**For all output variables, the untransformed dataset produces much lower RMSEs. Next, we examine whether hyperparameter tuning improves the output.**

#### Hyperparameter Tuning: 
#### Find Best Hyperparameters:

In [119]:
parametersRF = {"n_estimators":[int(x) for x in np.linspace(start = 1, stop = 10, num = 10)],
               "max_features": list(range(1,5)),
               "max_depth": list(range(1,15)),
               "min_samples_leaf": [3, 5, 7, 10, 15, 20],
               "bootstrap":[True, False]}

In [120]:
print((color.BOLD + "Random Forest: Best Hyperparameters" + color.END))
outputRFTuned=Tune(Untransformed_Dataset, RandomForestRegressor(random_state=1), parametersRF)
outputRFTuned

Random Forest: Best Hyperparameters


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Optimal Depth,Max Features,Min. Samples for Leaf Node,Best Feature,Best Feature Importance,Second Best Feature,Second Best Feature Importance
0,OCBD Income Return (%),[0.1617],[0.3158],4,1,7,JOBS_1000,0.420096,A_MEDIAN,0.191323
1,OSUB Income Return (%),[0.1975],[0.5187],5,2,3,TOT_EMP,0.430040,JOBS_1000,0.409559
2,Office Income Return (%),[0.1726],[0.5384],5,2,3,JOBS_1000,0.423670,TOT_EMP,0.385703
3,OCBD Apprec Return (%),[0.1973],[0.5401],6,1,3,TOT_EMP,0.255931,A_MEDIAN,0.245863
4,OSUB Apprec Return (%),[0.1974],[0.277],7,2,7,A_MEDIAN,0.273292,A_MEAN,0.232461
5,Office Apprec Return (%),[0.1824],[0.4369],9,1,3,A_MEAN,0.305599,JOBS_1000,0.228298
6,OCBD Total Return (%),[0.326],[0.5717],8,1,3,TOT_EMP,0.270931,JOBS_1000,0.212676
7,OSUB Total Return (%),[0.3032],[0.5512],7,1,3,JOBS_1000,0.316743,TOT_EMP,0.225059
8,Office Total Return (%),[0.3032],[0.5512],7,1,3,JOBS_1000,0.316743,TOT_EMP,0.225059


#### Ensemble Method: Ada-Boost
#### Find Best Dataset: Find Best Dataset (lowest RMSE): Original Dataset

In [121]:
agg_output_AB=pd.DataFrame()
for i in range(0, 2):
    to_appendAB=FFCVUntuned(DS_ls[i],All_Features, AdaBoostRegressor(random_state=1))
    agg_output_AB=agg_output_AB.append(to_appendAB)

In [122]:
print((color.BOLD + "Untuned Ada-Boost (Decision Tree) Output" + color.END))
agg_output_AB.sort_values(by="4-Fold Mean RMSE", ascending=True).reset_index(drop=True)

Untuned Ada-Boost (Decision Tree) Output


,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Best Feature,Feature Importance,Dataset
0,OCBD Income Return (%),[0.1562],[0.5282],JOBS_1000,0.31,Preprocessed Untransformed Dataset
1,Office Income Return (%),[0.175],[0.4992],JOBS_1000,0.41,Preprocessed Untransformed Dataset
2,OSUB Income Return (%),[0.1881],[0.516],JOBS_1000,0.33,Preprocessed Untransformed Dataset
3,OCBD Apprec Return (%),[0.195],[0.5805],JOBS_1000,0.34,Preprocessed Untransformed Dataset
4,Office Apprec Return (%),[0.198],[0.4551],JOBS_1000,0.32,Preprocessed Untransformed Dataset
5,OSUB Apprec Return (%),[0.2271],[0.3258],JOBS_1000,0.38,Preprocessed Untransformed Dataset
6,OCBD Total Return (%),[0.3385],[0.5358],JOBS_1000,0.33,Preprocessed Untransformed Dataset
7,Office Total Return (%),[0.3446],[0.3981],JOBS_1000,0.34,Preprocessed Untransformed Dataset
8,OSUB Total Return (%),[0.3446],[0.3981],JOBS_1000,0.34,Preprocessed Untransformed Dataset
9,OSUB Income Return (%),[0.647],[0.7011],A_MEAN,0.29,Log-transformed Dataset


#### Conclusion: Random Forest Modeling
#### Overall, it appears the best method is untuned Random Forest, evidenced by lowest RMSE and highest R2. 

In [123]:
agg_all_RF=pd.merge(pd.merge(agg_output_RF[0:9], outputRFTuned, on='Output Variable'),agg_output_AB[0:9], on='Output Variable')

In [124]:
agg_all_RF=agg_all_RF[["Output Variable", "4-Fold Mean RMSE_x","4-Fold Mean R2_x", 
            "4-Fold Mean RMSE_y","4-Fold Mean R2_y",
            "4-Fold Mean RMSE","4-Fold Mean R2"]]

In [125]:
agg_all_RF.rename(columns={'Output Variable':'Output Variable', '4-Fold Mean RMSE_x':'RF Untuned: 4-Fold Mean RMSE',
                 '4-Fold Mean R2_x':'RF Untuned: 4-Fold Mean R2',
                 '4-Fold Mean RMSE_y':'RF Tuned: 4-Fold Mean RMSE',
                 '4-Fold Mean R2_y':'RF Tuned: 4-Fold Mean R2',
                 '4-Fold Mean RMSE':'Ada-Boost: 4-Fold Mean RMSE',
                 '4-Fold Mean R2':'Ada-Boost:4-Fold Mean R2'},inplace=True)

In [126]:
agg_all_RF

,Output Variable,RF Untuned: 4-Fold Mean RMSE,RF Untuned: 4-Fold Mean R2,RF Tuned: 4-Fold Mean RMSE,RF Tuned: 4-Fold Mean R2,Ada-Boost: 4-Fold Mean RMSE,Ada-Boost:4-Fold Mean R2
0,OCBD Income Return (%),[0.1519],[0.6716],[0.1617],[0.3158],[0.1562],[0.5282]
1,OSUB Income Return (%),[0.1882],[0.698],[0.1975],[0.5187],[0.1881],[0.516]
2,Office Income Return (%),[0.171],[0.6924],[0.1726],[0.5384],[0.175],[0.4992]
3,OCBD Apprec Return (%),[0.1978],[0.6747],[0.1973],[0.5401],[0.195],[0.5805]
4,OSUB Apprec Return (%),[0.2116],[0.6254],[0.1974],[0.277],[0.2271],[0.3258]
5,Office Apprec Return (%),[0.185],[0.6733],[0.1824],[0.4369],[0.198],[0.4551]
6,OCBD Total Return (%),[0.3198],[0.6882],[0.326],[0.5717],[0.3385],[0.5358]
7,OSUB Total Return (%),[0.3215],[0.6549],[0.3032],[0.5512],[0.3446],[0.3981]
8,Office Total Return (%),[0.3215],[0.6549],[0.3032],[0.5512],[0.3446],[0.3981]


### Conclusion: Best Overall Model
---
#### Overall, it appears the best method is untuned Random Forest on preprocessed untransformed dataset, evidenced by lowest RMSE. The differential in R2 from XG Boost-untuned is not substantial.

In [127]:
agg_all=pd.merge(agg_all_DT, agg_all_RF, on='Output Variable')

In [128]:
agg_all

,Output Variable,DT Untuned: 4-Fold Mean RMSE,DT Untuned: 4-Fold Mean R2,DT Tuned: 4-Fold Mean RMSE,DT Tuned: 4-Fold Mean R2,XG Boost: 4-Fold Mean RMSE,XG Boost:4-Fold Mean R2,RF Untuned: 4-Fold Mean RMSE,RF Untuned: 4-Fold Mean R2,RF Tuned: 4-Fold Mean RMSE,RF Tuned: 4-Fold Mean R2,Ada-Boost: 4-Fold Mean RMSE,Ada-Boost:4-Fold Mean R2
0,OCBD Income Return (%),[0.2339],[0.4654],[0.1544],[0.3679],[0.1689],[0.719],[0.1519],[0.6716],[0.1617],[0.3158],[0.1562],[0.5282]
1,OSUB Income Return (%),[0.2397],[0.6487],[0.1981],[0.2777],[0.1965],[0.7625],[0.1882],[0.698],[0.1975],[0.5187],[0.1881],[0.516]
2,Office Income Return (%),[0.2383],[0.5752],[0.1822],[0.21],[0.1907],[0.7268],[0.171],[0.6924],[0.1726],[0.5384],[0.175],[0.4992]
3,OCBD Apprec Return (%),[0.2348],[0.6687],[0.2028],[0.1833],[0.1825],[0.8013],[0.1978],[0.6747],[0.1973],[0.5401],[0.195],[0.5805]
4,OSUB Apprec Return (%),[0.2706],[0.5299],[0.2001],[0.1616],[0.2326],[0.6578],[0.2116],[0.6254],[0.1974],[0.277],[0.2271],[0.3258]
5,Office Apprec Return (%),[0.2288],[0.609],[0.1976],[0.1513],[0.2036],[0.6814],[0.185],[0.6733],[0.1824],[0.4369],[0.198],[0.4551]
6,OCBD Total Return (%),[0.4472],[0.5747],[0.3391],[0.1463],[0.3283],[0.7707],[0.3198],[0.6882],[0.326],[0.5717],[0.3385],[0.5358]
7,OSUB Total Return (%),[0.3781],[0.6559],[0.3254],[0.145],[0.3426],[0.6981],[0.3215],[0.6549],[0.3032],[0.5512],[0.3446],[0.3981]
8,Office Total Return (%),[0.3781],[0.6559],[0.3254],[0.145],[0.3426],[0.6981],[0.3215],[0.6549],[0.3032],[0.5512],[0.3446],[0.3981]


### Conclusion: Most Important Features
---
#### The most Important features for most output variables is either <u>TOT_EMP</u> or <u>JOBS_1000</u>.

In [129]:
agg_output_RF[0:9]

,Output Variable,4-Fold Mean RMSE,4-Fold Mean R2,Best Feature,Feature Importance,Dataset
0,OCBD Income Return (%),[0.1519],[0.6716],TOT_EMP,0.37,Preprocessed Untransformed Dataset
1,OSUB Income Return (%),[0.1882],[0.698],TOT_EMP,0.36,Preprocessed Untransformed Dataset
2,Office Income Return (%),[0.171],[0.6924],TOT_EMP,0.33,Preprocessed Untransformed Dataset
3,OCBD Apprec Return (%),[0.1978],[0.6747],JOBS_1000,0.31,Preprocessed Untransformed Dataset
4,OSUB Apprec Return (%),[0.2116],[0.6254],TOT_EMP,0.30,Preprocessed Untransformed Dataset
5,Office Apprec Return (%),[0.185],[0.6733],JOBS_1000,0.34,Preprocessed Untransformed Dataset
6,OCBD Total Return (%),[0.3198],[0.6882],JOBS_1000,0.31,Preprocessed Untransformed Dataset
7,OSUB Total Return (%),[0.3215],[0.6549],JOBS_1000,0.38,Preprocessed Untransformed Dataset
8,Office Total Return (%),[0.3215],[0.6549],JOBS_1000,0.38,Preprocessed Untransformed Dataset


**Occupations with the five highest average 10-year annualized growth in <u>TOT_EMP</u> or <u>JOBS_1000</u> are:**

- Healthcare Support 
- Transportation and Material Moving 
- Management
- Computer and Mathematical 
- Business and Financial Operations<br>

**Corporate migrations in these industries have likely had the most impact on an MSA's office real estate value.**

In [141]:
Untransformed_Dataset.groupby("OCC_TITLE").mean().sort_values(by=["TOT_EMP", "JOBS_1000"], ascending=False).head(5)

,TOT_EMP,JOBS_1000,LOC_QUOTIENT,A_MEAN,A_MEDIAN,OCBD Income Return (%),OSUB Income Return (%),Office Income Return (%),OCBD Apprec Return (%),OSUB Apprec Return (%),Office Apprec Return (%),OCBD Total Return (%),OSUB Total Return (%),Office Total Return (%)
OCC_TITLE,,,,,,,,,,,,,,
Healthcare Support Occupations,5.915612,4.703372,0.638328,1.611433,1.506237,0.162237,0.214744,0.194360,0.163173,0.089362,0.120623,0.331081,0.307438,0.307438
Transportation and Material Moving Occupations,4.047680,2.858031,0.205522,2.198346,1.831702,0.312569,0.406998,0.366784,0.307072,0.161175,0.229946,0.630526,0.574289,0.574289
Management Occupations,3.675523,2.574955,0.659246,1.789217,1.880491,0.310667,0.363301,0.324730,0.287947,0.152326,0.205069,0.608533,0.521248,0.521248
Computer and Mathematical Occupations,3.495422,2.306887,-0.152850,2.295442,2.271399,0.223445,0.254038,0.226974,0.227450,0.116108,0.158647,0.458694,0.374648,0.374648
Business and Financial Operations Occupations,3.288395,2.103739,-0.207331,1.722843,1.716078,0.337440,0.398753,0.357355,0.323331,0.158678,0.227399,0.672029,0.563443,0.563443
